## Display Env Vars and Python Sys Path
Make sure all `$VAR`s have been expanded properly 

In [3]:
!export

export CLICOLOR='1'
export CONDA_DEFAULT_ENV='py3'
export GIT_PAGER='cat'
export HOME='/root'
export JPY_PARENT_PID='90'
export MPLBACKEND='module://ipykernel.pylab.backend_inline'
export PAGER='cat'
export PATH='/opt/conda/envs/py3/bin:/root/spark-2.0.1-SNAPSHOT-bin-fluxcapacitor/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin'
export PWD='/root/notebooks'
export PYSPARK_SUBMIT_ARGS='--master local[*] --packages databricks:tensorframes:0.2.3-s_2.10 --jars /root/lib/jpmml-sparkml-package-1.0-SNAPSHOT.jar --py-files /root/lib/jpmml.py pyspark-shell'
export PYTHONPATH='/root/spark-2.0.1-SNAPSHOT-bin-fluxcapacitor/python:/root/spark-2.0.1-SNAPSHOT-bin-fluxcapacitor/python/lib/py4j-0.10.3-src.zip'
export SHELL='/bin/bash'
export SPARK_HOME='/root/spark-2.0.1-SNAPSHOT-bin-fluxcapacitor'
export SPARK_MASTER_HOST='local[*]'
export SPARK_MASTER_PORT=''
export TERM='xterm-color'
export USER='root'


In [4]:
import sys
sys.path

['',
 '/root/spark-2.0.1-SNAPSHOT-bin-fluxcapacitor/python',
 '/root/spark-2.0.1-SNAPSHOT-bin-fluxcapacitor/python/lib/py4j-0.10.3-src.zip',
 '/opt/conda/envs/py2/lib/python27.zip',
 '/opt/conda/envs/py2/lib/python2.7',
 '/opt/conda/envs/py2/lib/python2.7/plat-linux2',
 '/opt/conda/envs/py2/lib/python2.7/lib-tk',
 '/opt/conda/envs/py2/lib/python2.7/lib-old',
 '/opt/conda/envs/py2/lib/python2.7/lib-dynload',
 '/opt/conda/envs/py2/lib/python2.7/site-packages/Sphinx-1.4.1-py2.7.egg',
 '/opt/conda/envs/py2/lib/python2.7/site-packages/setuptools-23.0.0-py2.7.egg',
 '/opt/conda/envs/py2/lib/python2.7/site-packages',
 '/opt/conda/envs/py2/lib/python2.7/site-packages/IPython/extensions',
 '/root/.ipython']

## Setup Spark and SQL Contexts

In [5]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext

sparkContext = SparkContext.getOrCreate()
sqlContext = SQLContext(sparkContext)

sqlContext

## Build Decision Tree (Regression) with Spark ML Pipeline

In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import RFormula
from pyspark.ml.regression import DecisionTreeRegressor

data = sqlContext.read.csv('file:///root/notebooks/data/wine.csv', header = True, inferSchema = True)

formula = RFormula(formula = "quality ~ .")
regressor = DecisionTreeRegressor()
pipeline = Pipeline(stages = [formula, regressor])
pipelineModel = pipeline.fit(data)

pipelineModel

PipelineModel_46ec805b9f24a240725f

## Convert Generated Spark ML Pipeline to PMML

In [7]:
from jpmml import toPMMLBytes

pmmlBytes = toPMMLBytes(sparkContext, data, pipelineModel)

str(pmmlBytes)

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<PMML xmlns="http://www.dmg.org/PMML-4_3" version="4.3">\n\t<Header>\n\t\t<Application/>\n\t\t<Timestamp>2016-09-26T03:47:39Z</Timestamp>\n\t</Header>\n\t<DataDictionary>\n\t\t<DataField name="quality" optype="continuous" dataType="double"/>\n\t\t<DataField name="volatile_acidity" optype="continuous" dataType="double"/>\n\t\t<DataField name="citric_acid" optype="continuous" dataType="double"/>\n\t\t<DataField name="residual_sugar" optype="continuous" dataType="double"/>\n\t\t<DataField name="chlorides" optype="continuous" dataType="double"/>\n\t\t<DataField name="free_sulfur_dioxide" optype="continuous" dataType="double"/>\n\t\t<DataField name="total_sulfur_dioxide" optype="continuous" dataType="double"/>\n\t\t<DataField name="density" optype="continuous" dataType="double"/>\n\t\t<DataField name="pH" optype="continuous" dataType="double"/>\n\t\t<DataField name="sulphates" optype="continuous" dataType="double"/>\n\t\t<DataField n